In [1]:
import datetime as dt
import time

from dateutil.relativedelta import relativedelta
from concurrent import futures

from findStocks import *

In [2]:
# Get user inputs
# stockFile, dataFolder, apikey, flexible, record = getInputs()

In [3]:
now = dt.datetime.now()
lastday = now + relativedelta(months=0, days=-2)
end_date = lastday.strftime('%Y-%m-%d')

In [4]:
data_dir = "../data"
stockFile = f"{data_dir}/SP500_{end_date}.csv"
dataFolder = "../data/historical"
apikey = "P6QVC3IFRGHVKGAN"
flexible = ""

In [5]:
symbols = getData(stockFile, dataFolder, apikey)

In [ ]:
# Loop through all stocks
for s in symbols:
    print('>>>', s)

    try:
        # Initialize Stock object
        stock = Stock(s, dataFolder, flexible)

        # Run preliminary tests to check if stock is disqualified
        preliminaryTests(stock)

        # Skip stock if it failed preliminary tests. Update the *Processed.csv
        if stock.errorMessage != 'processed':
            updateProcessed(stockFile, s, stock.errorMessage)
            continue

        for r in stock.reports:
            # Manage missing data and other inconsistencies in data
            manageBadData(stock, r)

            # Calculate new metrics and run stock tests
            analyzeStock(stock, r)

        # Save data and update files
        record = saveResults(stock, s, stockFile, record)
        
        # Assert can occur when all stocks fail preliminary tests
        assert record != [], 'No procssed stocks to save to *Results.csv'

        # Save the test results for all stocks
        columns = [x for x in stock.record.keys()]
        saveAll(record, columns)

    except:
        pass

In [ ]:
# Loop through all stocks
for s in symbols:

In [19]:
st = time.time()
# Option1) MultiTreading
# in case a smaller number of stocks than threads was passed in
max_workers = 2
workers = min(max_workers, len(symbols))

with futures.ThreadPoolExecutor(workers) as executor:
    for number, record in zip(symbols, executor.map(testStock, symbols, dataFolder, flexible)):
        print('%d is record: %s' % (number, record))

# Option2) MultiProcessing
#with futures.ProcessPoolExecutor() as executor:
#    for stock, record in zip(symbols, executor.map(testStock, symbols, dataFolder, flexible)):
#        # Assert can occur when all stocks fail preliminary tests
#        assert record != [], 'No procssed stocks to save to *Results.csv'

#        # Save the test results for all stocks
#        columns = [x for x in stock.record.keys()]

et = time.time() - st

print(f'Time: {et}')
# Assert can occur when all stocks fail preliminary tests

Time: 0.0


In [11]:
columns

NameError: name 'columns' is not defined